<a href="https://colab.research.google.com/github/gassis/regifner-br/blob/master/docs_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install required packages
!pip install "unstructured[all-docs]"
!pip install PyPDF2
!pip install pymupdf
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit weaviate-client langchain sentence-transformers tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 70

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.6 MB

In [2]:
# fixing unicode error in google colab
import locale

locale.getpreferredencoding = lambda: "UTF-8"

# import dependencies
from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf
import os
import weaviate
import json
import torch
import io
import tempfile
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Weaviate
import weaviate
import re, requests
from bs4 import BeautifulSoup as BS
from bs4 import Comment
from langchain_community.document_loaders import PyPDFLoader
import urllib.parse

In [3]:
ifg_url = "http://www.ifg.edu.br"
ifg_docs_url = "https://www.ifg.edu.br/documentos?showall=&limitstart="

In [4]:
docs = []
s = requests.Session()
try:
  r = s.get('https://www.ifg.edu.br/documentos?showall=&limitstart=', verify=False)
  response = r.text
  soup = BS(response, 'html.parser')
  for link in soup.select("p > a"):
    link = str(link.get_attribute_list('href')[0])
    if link == "http://www.ifg.edu.br/":  continue
    if not link.startswith("http"):
      link = ifg_url+link
    r = s.get(link, verify=False)
    response = r.text
    soup = BS(response, 'html.parser')
    files = soup.find_all(href=re.compile(".pdf"))
    for file in files:
      link = str(file.get('href'))
      if not link.startswith("http"):
        link = ifg_url+link
      docs.append(link)
  r = s.get('https://www.ifg.edu.br/boletim-de-servico', verify=False)
  response = r.text
  soup = BS(response, 'html.parser')
  for link in soup.find_all("a", class_="toclink"):
    link = str(link.get('href'))
    if not link.startswith("http"):
      link = ifg_url+link
    r = s.get(link, verify=False)
    response = r.text
    soup = BS(response, 'html.parser')
    files = soup.find_all(href=re.compile(".pdf"))
    for file in files:
      link = str(file.get('href'))
      if not link.startswith("http"):
        link = ifg_url+link
      docs.append(link)
except requests.Timeout as err:
  print(err.message)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ifg.edu.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ifg.edu.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
print(docs[0])

http://www.ifg.edu.br/attachments/article/11547/Estatuto_IFG_2018.pdf


In [6]:
s = requests.Session()
try:
  r = s.get(docs[0], verify=False)
  pdf_io_bytes = io.BytesIO(r.content)
  with tempfile.NamedTemporaryFile(delete=False) as atemp:
    atemp.write(pdf_io_bytes.getvalue())
except requests.Timeout as err:
  print(err.message)

In [7]:
#from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredFileLoader

#loader = PyPDFLoader(atemp.name)
loader = UnstructuredFileLoader(atemp.name)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
docs

[Document(page_content='MINISTÉRIO DA EDUCAÇÃO SECRETARIA DE EDUCAÇÃO PROFISSIONAL E TECNOLÓGICA INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE GOIÁS\n\nESTATUTO\n\nINSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE GOIÁS – IFG\n\nRevisado durante os debates do Congresso Institucional IFG 2018\n\n1\n\nMINISTÉRIO DA EDUCAÇÃO SECRETARIA DE EDUCAÇÃO PROFISSIONAL E TECNOLÓGICA INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE GOIÁS\n\nTÍTULO I DA INSTITUIÇÃO\n\nCAPÍTULO I DA NATUREZA E DAS FINALIDADES\n\nArt. 1º O Instituto Federal de Educação, Ciência e Tecnologia de Goiás (IFG), instituição criada nos termos da Lei Nº. 11.892 de 29 de dezembro de 2008, vinculada ao Ministério da Educação, possui natureza jurídica de autarquia, sendo detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar.\n\n§ 1º O Instituto Federal de Educação, Ciência e Tecnologia de Goiás é uma instituição de educação superior, básica e profissional, pluricurr

## Load documents into Weaviate

*   Item da lista
*   Item da lista



With the documents preprocessed, we're now ready to load them into ChromaDB. We accomplish this easily by using the OpenAI embeddings the Chroma vectrostore from `langchain`. This workflow will vectorize the documents using the OpenAI embeddings endpoint, and then load the documents and associated vectors into Chroma. Once the documents are in Chroma, we can perform a similarity search to retrieve documents related to our topic of interest.

In [9]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [10]:
WEAVIATE_URL = "https://ged-lehw4wcj.weaviate.network"
WEAVIATE_API_KEY = "RDK6txG5eLtf0TRczkROMFPy6pno6fF5P4QK"

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [11]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

In [12]:
query_docs = vector_db.similarity_search("Qual é o regime disciplinar do IFG?", k=3)

In [13]:
query_docs

[Document(page_content=' do cumprimento da legislação interna e externa que regula o funcionamento do IFG, dentro dos limites e possibilidades estabelecidas pelo CONSUP. É também responsável pelo apoio, dentro de suas especificidades, aos Órgãos do Sistema de Controle Interno do poder Executivo e ao Tribunal de Contas da União, respeitadas a legislação pertinente.\n\nParágrafo único', metadata={'source': '/tmp/tmpvb1ji98l'}),
 Document(page_content=' 55 O regime disciplinar do corpo docente e do corpo técnico-administrativo do Instituto Federal de Goiás observa as disposições legais, as normas e os regulamentos sobre a ordem disciplinar e as sanções aplicáveis, bem como os recursos cabíveis, previstos pela legislação federal.\n\n16\n\nMINISTÉRIO DA EDUCAÇÃO SECRETARIA DE EDUCAÇÃO PROFISSION', metadata={'source': '/tmp/tmpvb1ji98l'}),
 Document(page_content=' Câmpus (Concâmpus) é a instância máxima do Câmpus que, em conformidade com os princípios expressos na legislação do IFG, no plane

In [14]:
# specify model huggingface mode name
model_name = "maritaca-ai/sabia-7b"
#model_name = "anakin87/zephyr-7b-alpha-sharded"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)
# load model
model = load_quantized_model(model_name)
# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

In [ ]:
from datetime import datetime
start_time = datetime.now()
response = qa_chain.run("Qual é a sigla do Instituto Federal de Educação, Ciência e Tecnologia de Goiás?")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(response)